<a href="https://colab.research.google.com/github/RobDrie/IT-Tools-Spark/blob/main/Spark_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=e3c63bc4f9f480def222ef472f16a6958eef16b9245f8178e9b6f36a7ede6844
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import pyspark

In [3]:
spark = pyspark.sql.SparkSession.builder.appName('Session1').getOrCreate()

## Load & Process gas price data with Spark

In [4]:
!git clone https://github.com/rvm-courses/GasPrices

Cloning into 'GasPrices'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 143 (delta 3), reused 0 (delta 0), pack-reused 124
Receiving objects: 100% (143/143), 619.51 MiB | 28.59 MiB/s, done.
Resolving deltas: 100% (33/33), done.
Updating files: 100% (40/40), done.


In [6]:
!ls

drive  GasPrices  sample_data


In [8]:
data = spark.read.csv('GasPrices/Prix2019.csv.gz', inferSchema=True)